In [63]:
import os
import json
import requests

In [64]:
MOVIELENS_HOST = os.environ.get("MOVIELENS_HOST", "localhost")
MOVIELENS_SCHEMA = os.environ.get("MOVIELENS_SCHEMA", "http")
MOVIELENS_PORT = os.environ.get("MOVIELENS_PORT", "5000")

MOVIELENS_USER = os.environ.get("MOVIELENS_USER", "airflow")
MOVIELENS_PASSWORD = os.environ.get("MOVIELENS_PASSWORD", "airflow")

In [65]:
def _get_session():
    session = requests.Session()
    session.auth = (MOVIELENS_USER, MOVIELENS_PASSWORD)  # Replace with your credentials

    base_url = f"{MOVIELENS_SCHEMA}://{MOVIELENS_HOST}:{MOVIELENS_PORT}"

    return session, base_url

In [66]:
session, base_url = _get_session()

In [67]:
session

In [68]:
base_url

'http://localhost:5000'

In [ ]:
def _get_with_pagination(session, url, params, batch_size=100):
    
    offset = 0
    total = None
    while total is None or offset < total:
        response = session.get(url,
                               params={
                                   **params,
                                   **{"offset": offset, "limit": batch_size}
                               })
        response.raise_for_status()
        response_json = response.json()

        yield from response_json["result"] # list

        offset += batch_size
        total = response_json["total"]

In [ ]:
def _get_ratings(start_date, end_date, batch_size=100):
    session, base_url = _get_session()

    yield from _get_with_pagination(  # generator
        session=session,
        url=f"{base_url}/ratings",
        params={
            "start_date": start_date,
            "end_date": end_date,
        },
        batch_size=batch_size,
    )

In [162]:
a1 = _get_ratings("2019-01-01", "2019-01-02")
a1

<generator object _get_ratings at 0x7eb048fb5e40>

In [165]:
next(a1)

{'movieId': 55820, 'rating': 4.0, 'timestamp': 1546301706, 'userId': 27667}

In [166]:
next(a1)

{'movieId': 2329, 'rating': 4.0, 'timestamp': 1546301788, 'userId': 27667}